In [ ]:
import os
os.environ["OPENAI_API_KEY"] = ""

In [ ]:
# Define your input folder langchain GitHub files and the persist directory for ChromaDB
input_folder = "./langchain-master"
persist_directory = 'db_langchain_github'

In [ ]:
# Load every file from the folder
import os
from langchain.document_loaders import TextLoader

root_dir = './langchain-master'
docs = []
for dirpath, dirnames, filenames in os.walk(root_dir):
    for file in filenames:
        try: 
            loader = TextLoader(os.path.join(dirpath, file), encoding='utf-8')
            docs.extend(loader.load_and_split())
        except Exception as e: 
            pass

In [ ]:
# Split loaded files to chunks
from langchain.text_splitter import CharacterTextSplitter

text_splitter = CharacterTextSplitter(chunk_size=1250, chunk_overlap=10)
texts = text_splitter.split_documents(docs)

In [ ]:
# Initialize Chrome data base 
def init_chromadb(documents, persist_directory):
    client_settings = chromadb.config.Settings(
        chroma_db_impl="duckdb+parquet",
        persist_directory=persist_directory,
        anonymized_telemetry=False
    )
    embeddings = OpenAIEmbeddings(disallowed_special=())

    vectorstore = Chroma(
        collection_name="langchain_github",
        embedding_function=embeddings,
        client_settings=client_settings,
        persist_directory=persist_directory,
    )

    vectorstore.add_documents(documents=documents, embedding=embeddings)
    vectorstore.persist()

In [ ]:
# Initialize embeddings and create Chroma database
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
import chromadb

init_chromadb(texts, persist_directory)